The basic process for an interpreter is 

1. Read the next expression from a stream of characters: like a string or a file.
2. Evaluate the expression according to the rules of the language.
3. Print the result of the evaluated expression.
4. Loop back to 1.

This is is known as the Read-Eval-Print-Loop or REPL for short.

We'll start with implementing Scheme's read.

In [1]:
def read(stream): 
    return read_from_token(stream, take_token(stream))

`read` takes a stream of characters. This could be a string or a file -- something that we can take characters from.
It passes that stream to `take_token`, which will take the next interesting token (or substring) from the stream.
It passes the stream and token to `read_from_token`, which reads and returns the expression based on the token.

In [2]:
def read_from_token(stream, token):
    if token in lisp_token_table: return lisp_token_table[token](stream)
    elif token: return read_default_token(stream, token)
    else: error("End of stream encountered")

In [3]:
def error(string): raise Exception(string)

`read_from_token`'s job is to dispatch to a `lisp_token_table` dictionary. This dictionary has tokens associated with functions to read expressions based on those tokens.
If the token isn't in the dictionary, it's assumed to be a default token. 

Tokens in the `lisp_token_table` might be things like `(` `"` `;`, etc., while default tokens are more flexible like `name-of-variable` or `3.14159`.

The token table allows us to easily and flexibly add more tokens later on, without redefining any functions. We will take advantage of this by adding token-readers later.

In [4]:
lisp_token_table = {}

A default token is either a number, or, if Python doesn't recognize it as a number, a symbol.

In [5]:
def read_default_token(stream, token): return number_or_symbol_from_token(token)

`take_token` takes characters from a stream, discarding uninteresting characters (whitespace) and returning the next interesting Scheme substring.

In [6]:
def take_token(stream):
    discard_while(stream, is_lisp_delimiter) # discard uninteresting characters
    return take_token_here(stream, lisp_escape_character, is_lisp_delimiter, lisp_delimiting_tokens)

Scheme tokens are characterized by `lisp_delimiting_tokens`, `is_lisp_delimiter`, and the `lisp_escape_character`.
They are defined outside of `take_token`, so that we can extend/modify them later.

In Scheme, `()` enclose a list, `;`, `#|`, and `#;` all specify different kinds of comments, and `"` specifies a string of text. We will add more later.

In [7]:
lisp_delimiting_tokens = [ '(', ')', ';', '#|', '#;', '"' ]

In addition to delimiting tokens, Scheme is separated by whitespace.

In [8]:
def is_lisp_delimiter(char): return is_whitespace(char)

In [9]:
def is_whitespace(char): return char.isspace()

In any normal situation a `\` will escape the next character in Scheme text. This behavior occurs in and outside of strings.

In [10]:
lisp_escape_character = '\\'

`take_token_here` assumes it is at the start of an interesting token, and takes characters from the stream until the end of the token is encountered.

If the next token in the stream is a delimiting token, it is taken and returned.
Otherwise, a default token is taken until the end of token is encountered.
For Scheme the end of the token is either a whitespace character (`is_whitespace` is passed to `is_delimiter_character`), or a delimiting token (such as a `"`, `(`, or `)`), as listed in `lisp_delimiting_tokens`. 

In [11]:
def take_token_here(stream, escape_character, is_delimiter_character, delimiting_tokens):
    def is_end_of_token(stream):
        # if the next character is a delimiter character (like whitespace for Scheme)
        if is_delimiter_character(stream.peek()): return True
        # Otherwise, see if the start of the stream is one of the delimiting_tokens
        # using stream save/restore to avoid taking characters from the stream
        position = stream.save()
        delimiting_token = take_prefix_match(stream, delimiting_tokens)
        stream.restore(position)
        return delimiting_token is not None

    if stream.is_eos(): return None
    # if the next token is one of the delimiting_tokens...
    token = take_prefix_match(stream, delimiting_tokens)
    if token: return token
    # Otherwise, it's a default token
    return take_until_escaping(stream, escape_character, is_end_of_token)

A stream could be a StringStream or a FileStream or a NetworkStream etc.

The term `eos` denotes "end of stream" like `eof` used like denotes "end of file".

Streams take one character at a time (using `next`), but `peek` can be used to look at the next character without taking it.
`save` and `restore` are used to read ahead without permanently taking characters.

A StringStream is basically a string with a position/index into the string. The position updates whenever `next` is called.

In [12]:
class StringStream:
    def __init__(self, string):
        self.string = string
        self.length = len(string)
        self.position = 0
    
    # Is End of stream
    def is_eos(self): return self.position == self.length
    def peek(self): return None if self.is_eos() else self.string[self.position]
    def next(self):
        char = self.peek()
        if char: self.position += 1
        return char
    
    def save(self): return self.position
    def restore(self, position): self.position = position

In [13]:
test_stream = StringStream('The quick brown fox')
print(test_stream.peek())
test_string = ''
while True:
    test_char = test_stream.next()
    if not test_char: break
    test_string += test_char
print(test_string)
test_stream.is_eos()

T
The quick brown fox


True

The escape character (`\` for Scheme) is used to escape the behavior of the following character, and instead treat it like a normal character.
For example, the string: `"I am about to \"quote\" this word"` escapes the behavior of `"` (which is to start/end a string) and instead treats it like it's any other character.

`take_until_escaping` takes characters from the given stream until `stream_predicate` returns true. It respects the power of the escape character.

`take_until_escaping` uses a stream_predicate, which can look ahead as many characters as needed, but _does not take any characters_.

In [14]:
def take_until_escaping(stream, escape_character, stream_predicate):
    string = ''
    while not stream.is_eos():
        # look at but don't take the next character
        char = stream.peek()
        if char == escape_character: 
            stream.next() # discard the escape_character
            escaped_character = stream.next()
            # Treat the escaped_character as a normal character
            # and append it to the string
            if escaped_character: string += escaped_character
        # if stream_predicate returns true, we are finished.
        elif stream_predicate(stream): return string
        # Otherwise we have a normal character: append it to the string
        else: string += stream.next()
    # we reached end of stream, return the string
    return string

To test `take_until_escaping` we'll use `character_predicate_to_stream_predicate` to take a character->boolean function and convert it to functions of stream->boolean.

In [15]:
def character_predicate_to_stream_predicate(character_predicate): return lambda stream: character_predicate(stream.peek())

In [16]:
test_stream = StringStream('A\\ Token\\ With\\ Escaped\\ Whitespace\nAfter')
# Take until we encounter a whitespace character
test_stream_predicate = character_predicate_to_stream_predicate(is_whitespace)
print(take_until_escaping(test_stream, '\\', test_stream_predicate))
test_stream.string[test_stream.position:]

A Token With Escaped Whitespace


'\nAfter'

`take_token` uses `discard_while` to discard uninteresting characters from the stream. Discarding does _not_ respect the escape character, but we can still implement it in terms of `take_until_escaping`, we just pass an empty string for the escape character.

In [17]:
def discard_while(stream, character_predicate):
    stream_predicate = character_predicate_to_stream_predicate(character_predicate)
    not_stream_predicate = complement(stream_predicate)
    take_until_escaping(stream, '', not_stream_predicate)

We use the `complement` of the given character predicate, which returns true whenever the given predicate would return false and vice versa.

In [18]:
def complement(f): return lambda *args: not f(*args)

In [19]:
test_stream = StringStream('    \n\t  The quick brown fox.')
discard_while(test_stream, is_whitespace)
test_stream.string[test_stream.position:]

'The quick brown fox.'

`take_prefix_match` is used by `take_token_here` to see if the stream is looking at a delimiting token. 

`take_prefix_match` tries to do an exact string match of each prefix in the list of prefixes. It takes and returns the first match. It uses stream's `save` and `restore` to backtrack if a match doesn't work out. If no matches are found, no characters are taken from the string, and `None` is returned.

In [20]:
def take_prefix_match(stream, prefixes):
    for prefix in prefixes:
        # Save the position of the stream
        position = stream.save()
        # take the number of characters in the prefix
        stream_prefix = take_n_characters(stream, len(prefix))
        # if they match, return the taken prefix
        if prefix == stream_prefix: return prefix
        # No match?: restore the position of the stream, and try the next prefix
        stream.restore(position)
    # No Matches found, return None
    return None

`take_n_characters` takes a given number of characters from the start of stream.

In [21]:
def take_n_characters(stream, num_characters):
    string = ''
    while not stream.is_eos() and len(string) < num_characters:
        char = stream.next()
        if char: string += char
    return string

In [22]:
take_n_characters(StringStream('01234'), 3)

'012'

In [23]:
take_n_characters(StringStream('01234'), 10)

'01234'

In [24]:
def test_take_prefix_match(stream, prefixes):
    token = take_prefix_match(stream, prefixes)
    return (token, stream.string[stream.position:])

In [25]:
test_take_prefix_match(StringStream('The quick brown fox'), ['The'])

('The', ' quick brown fox')

In [26]:
test_take_prefix_match(StringStream('The quick brown fox'), ['The quibble', 'The'])

('The', ' quick brown fox')

In [27]:
test_take_prefix_match(StringStream('The quick brown fox'), ['The quibbble'])

(None, 'The quick brown fox')

`take_prefix_match` always chooses the first prefix found:

In [28]:
test_take_prefix_match(StringStream('The quick brown fox'), ['The', 'Th', 'T'])

('The', ' quick brown fox')

If a conflict occurs between tokens, we probably want to take the longer token. Think about `"` vs. `"""`. If we always favor `"`, we will never be able to read `"""`.

We'll make sure to keep our tokens sorted:

In [29]:
def register_lisp_delimiting_token(token):
    lisp_delimiting_tokens.append(token)
    lisp_delimiting_tokens.sort(reverse=True, key=len)
lisp_delimiting_tokens.sort(reverse=True, key=len)
lisp_delimiting_tokens

['#|', '#;', '(', ')', ';', '"']

In [30]:
def test_take_token(string):
    stream = StringStream(string)
    return (take_token(stream), stream.string[stream.position:])

In [31]:
test_take_token('(a hullaballoo)')

('(', 'a hullaballoo)')

`take_token` discards delimiter characters until the next interesting token:

In [32]:
test_take_token('   #| a comment |#')

('#|', ' a comment |#')

Escape characters are respected, so whitespace _can_ be part of a symbol:

In [33]:
test_take_token('    a\\ symbol-with-whitespace another-symbol')

('a symbol-with-whitespace', ' another-symbol')

Here, `)` marks the end of `a-symbol` because it is a delimiting token.

In [34]:
test_take_token('a-symbol)')

('a-symbol', ')')

In [35]:
test_take_token('#;(an expression based comment token')

('#;', '(an expression based comment token')

We let Python do the heavy lifting of trying to convert the token to a number. If it fails. we assume it's a symbol and `intern` it into the symbol table.

In [36]:
def number_or_symbol_from_token(token):
    # try parsing as an int, float, complex in that order
    # if all three fail, it's a symbol
    try: return int(token)
    except ValueError:
        try: return float(token)
        except ValueError:
            try: return complex(token)
            except ValueError:
                # Remove the \s before storing the name 
                return intern_symbol(token)

Symbols are a lot like identifiers. They are more or less just names for variables, constants, values, and keywords.

In [37]:
class Symbol:
    def __init__(self, name): self.name = name
    def __repr__(self): return 'Symbol("' + self.name + '")'

When we `intern` a symbol, we store it in a symbol table (keys are names, values are symbols). This way we have one unique symbol object for any given symbol name, enabling fast pointer comparisons vs. string comparisons.

In [38]:
symbol_table = {}

def intern_symbol(name):
    global symbol_table
    # If a symbol exists with this name, return it
    if name in symbol_table: return symbol_table[name]
    # Otherwise, create a new symbol and store it in the symbol table
    symbol = Symbol(name)
    symbol_table[name] = symbol
    return symbol

Two interned symbols can be compared with `is`.

In [39]:
intern_symbol('foo') is intern_symbol('foo')

True

If one symbol is not interned, though:

In [40]:
intern_symbol('foo') is Symbol('foo')

False

Our read function now works for default tokens. We can read numbers:

In [41]:
read(StringStream('3.14159'))

3.14159

In [42]:
read(StringStream('3+4j'))

(3+4j)

And symbols:

In [43]:
read(StringStream('a-symbol'))

Symbol("a-symbol")

But:

In [44]:
read(StringStream('"a string"'))

Symbol(""")

`take_token` is able to tell us that `"` is an interesting substring, but `read` doesn't really know what to do with it. `"` isn't a symbol, but rather we want to read a string of text. We can teach it using our `lisp_token_table`.

In [45]:
lisp_token_table['"'] = lambda stream: lisp_read_string(stream)

`lisp_read_string` takes a stream and returns a string up until the next, enclosing, unescaped `"`.

In [46]:
def lisp_read_string(stream):
    def take_token(stream): return take_token_delimited(stream, ['"'])
    token = take_token(stream)
    # take and discard the quote token.
    quote_token = take_token(stream)
    if not quote_token == '"': error("End of stream while scanning string literal.")
    return token

`take_token_delimited` takes a token from stream up until one of the `token_delimiters` is encountered. In the case of `lisp_read_string`, it's looking for a close `"`.

Conveniently, we can reuse `take_token_here` to implement this.

In [47]:
def take_token_delimited(stream, token_delimiters):
    return take_token_here(stream, lisp_escape_character, constantly(False), token_delimiters)

We want to keep all of the string characters (i.e. don't ignore whitespace inside of a string), so we use `constantly(False)` to say that no characters are delimiter characters.

In [48]:
def constantly(value): return lambda _: value

In [49]:
test_stream = StringStream('"This \\"is\\" inside a quote"after')
(read(test_stream), test_stream.string[test_stream.position:])

('This "is" inside a quote', 'after')

If we reach the end of stream without finding the close quote:

In [50]:
try: read(StringStream('"Oops, no close quote'))
except Exception as e: print(str(e))

End of stream while scanning string literal.


Let's recognize some more tokens. Starting with line comments, which start with a `;` and go to the next `\n` newline character.

In [51]:
lisp_token_table[';'] = lambda stream: lisp_read_line_comment(stream)

When we read a comment, we discard the commented characters. But we aren't done, since we need to return an expression, we'll go ahead and call `scheme_read` _again_.

Like strings, we can implement this in terms of `take_token_delimited`. _Very_ convenient.

In [52]:
def lisp_read_line_comment(stream):
    # discard characters until the next (unescaped) newline
    take_token_delimited(stream, ['\n'])
    # Return the next read in expression.
    return read(stream)

In [53]:
read(StringStream('; a line comment\n"followed by a string"'))

'followed by a string'

In Scheme, block comments start with `#|` and go to the next _matching_ `|#`. Block comments can be nested, so they are a tiny bit more complex.

In [54]:
lisp_token_table['#|'] = lambda stream: lisp_read_block_comment(stream)

We use a count `num_blocks_open` to determine how many nested block comments we are in. If we encounter a `#|` the number goes up. If we encounter a closing `|#`, the number goes down. When we reach 0, we end the comment and `scheme_read` the next expression.

In [55]:
def lisp_read_block_comment(stream):
    def take_token(stream): return take_token_delimited(stream, ['#|', '|#'])
    num_open_blocks = 1
    # loop until num_open_blocks is 0 (or we reach end of stream)
    while True:
        token = take_token(stream)
        if   token == '#|': num_open_blocks += 1
        elif token == '|#':        
            num_open_blocks -= 1
            # if we closed all the blocks
            if num_open_blocks == 0: return read(stream)
        elif not token: error("End of stream encountered while reading block comment")

In [56]:
read(StringStream('#| outer #| inner #| innermost \|# gotcha |# |# |#"after"'))

'after'

Expression comments comment out the next expression. They start with `#;` and discard the next expression read in by `read`.

In [57]:
lisp_token_table['#;'] = lambda stream: lisp_read_expression_comment(stream)

In [58]:
def lisp_read_expression_comment(stream):
    read(stream) # Discard the commented expression
    return read(stream)

In [59]:
read(StringStream('#;"A commented expression" "an uncommented expression"'))

'an uncommented expression'

Now we come to lists (Lisp's bread and butter). There are two types:
- `Proper` Lists are started with a `(` followed by zero or more expressions, followed by a `)`.
    - Examples: `()`, `(1 2 3)`, `((1 2 3) (4 5 6) ((7 8 9) (10 11 12)))`
- `Dotted` Lists are started with a `(` followed by _one_ or more expressions, followed by a `.`, followed by exactly one expression, followed by a `)`.
    - Examples: `(1 . 2)` `(1 . (2 . (3 . 4)))` `(1 2 3 4 5 . 6)`

In [60]:
lisp_token_table['('] = lambda stream: lisp_read_list(stream)

Outside of a list, it's invalid to encounter a `)` or a `.`.

In [61]:
lisp_token_table[')'] = lambda stream: error("Read unexpected token ')' when outside of a list.")
lisp_token_table['.'] = lambda stream: error("read unexpected token '.' when outside of a list.")

Interestingly `.` is a token, but it is not a _delimiting_ token. That means `.` is _not_ a token when it is part of another symbol. 

In [62]:
def lisp_read_list(stream):
    def eos_error(): error("Encountered end of stream while reading list.")
    # a list of the expressions read in so far.
    expressions = []
    while not stream.is_eos():
        token = take_token(stream)
        # End of dotted list: (expression expressions... . last_expression)
        if   token == '.':
            # exactly one more expression expected
            last_expression = read(stream)
            close_list_token = take_token(stream)
            if not close_list_token: eos_error()
            if not close_list_token == ')': error("Expected end of list token after last expression in dotted list. Got: '" + close_list_token + "'")
            # return a dotted list
            return python_list_to_lisp_dotted_list(expressions, last_expression)
        # End of proper list: (expressions...)
        elif token == ')':
            return python_list_to_lisp_list(expressions)
        
        # token is an expression or part of an expression: read from the token
        expression = read_from_token(stream, token)
        expressions.append(expression)
    eos_error()

`lisp_read_list` does its own dispatch on tokens, so that it can handle `)` and `.`. It splits up the work of `read`, and does `take_token` and `read_from_token` on its own. _Veeeeeery_ convenient.

Even more fundamental than lists are pairs (`cons`es in Lisp parlance). Lists and dotted lists are built up out of these pairs.

In [63]:
def python_list_to_lisp_dotted_list(list, last_element):
    if len(list) == 0: error("Attempt to construct dotted list with no initial elements")
    result = last_element
    # Build the result in reverse order
    for x in reversed(list):
        result = Pair(x, result)
    return result

Proper lists are actually just dotted lists, whose dotted element is an empty list. E.g. `(1 2 3 . ())` is the same as `(1 2 3)`

In [64]:
def python_list_to_lisp_list(list):
    if len(list) == 0: return empty_list()
    return python_list_to_lisp_dotted_list(list, empty_list())

An empty list `()` is a list with zero elements (and therefore can't be represented as a pair). 

The actual Pair/EmptyList data types are trivial.

In [65]:
class EmptyList:
    def __repr__(self): return 'EmptyList()'
the_empty_list = EmptyList()
def empty_list(): return the_empty_list

Having a single EmptyList `the_empty_list` makes equality check easier.

In [66]:
class Pair:
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def __repr__(self): return 'Pair(' + repr(self.a) + ', ' + repr(self.b) + ')'

We can sort of see our reader works with lists now...

In [67]:
read(StringStream('(a b (c d . e) f g h ())'))

Pair(Symbol("a"), Pair(Symbol("b"), Pair(Pair(Symbol("c"), Pair(Symbol("d"), Symbol("e"))), Pair(Symbol("f"), Pair(Symbol("g"), Pair(Symbol("h"), Pair(EmptyList(), EmptyList())))))))

But the output is a bit noisy. To fix this, let's implement a `lisp_string` function which we can use to print Scheme text versions of objects to strings.

In [68]:
def lisp_string(object):
    t = type(object)
    if t in lisp_string_table: return lisp_string_table[t](object)
    # Otherwise just use the python printer
    else: return str(object)

Once again, we're using a dispatch table. `lisp_string_table` returns a function from object->string given the type of the object.

In [69]:
lisp_string_table = {}

In [70]:
lisp_string_table[str] = lambda string: string_to_lisp_string(string)

In [71]:
def string_to_lisp_string(string):
    def is_escape_char(char): return char in [lisp_escape_character, '"']
    return '"' + escape_string(string, is_escape_char) + '"'

Scheme strings need to escape any internal `"`s and any internal `\`s. We'll use `escape_string` to add those in.

In [72]:
def escape_string(string, is_escaped_char):
    result = ''
    for char in string:
        if is_escaped_char(char): result += lisp_escape_character
        result += char
    return result

In [73]:
print(escape_string('Here is a string with whitespace escaped.', is_whitespace))

Here\ is\ a\ string\ with\ whitespace\ escaped.


In [74]:
print(lisp_string(read(StringStream('"A string with \\"quotes\\" and \\\\backslashes\\\\"'))))

"A string with \"quotes\" and \\backslashes\\"


Most numbers can be printed with the default `str`...

In [75]:
lisp_string(read(StringStream('3.14159')))

'3.14159'

But Python chooses to wrap complex numbers with `()`s. This is a problem for Scheme!

In [76]:
lisp_string(read(StringStream('3+4j')))

'(3+4j)'

In [77]:
# If it is complex, we need to strip off the outer ()s that Python adds
lisp_string_table[complex] = lambda number: str(number)[1:-1]

In [78]:
lisp_string(read(StringStream('3+4j')))

'3+4j'

Problem solved.

Symbols, like strings, also need to escape some internal characters.

In [79]:
lisp_string_table[Symbol] = lambda symbol: symbol_to_lisp_string(symbol)

In [80]:
def symbol_to_lisp_string(symbol):
    delimiting_token_characters = lisp_delimiting_token_characters()
    def is_escaped_char(char):
        return char == lisp_escape_character or is_whitespace(char) or char in delimiting_token_characters
    return escape_string(symbol.name, is_escaped_char)

We start by collecting all characters that appear in delimiting tokens: when printing symbols, we want to escape all of these characters in addition to `\` and whitespace.

In [81]:
def lisp_delimiting_token_characters():
    delimiting_token_characters = []
    for token in lisp_delimiting_tokens:
        for char in token:
            if char not in delimiting_token_characters: 
                delimiting_token_characters.append(char)
    return delimiting_token_characters

In [82]:
lisp_delimiting_token_characters()

['#', '|', ';', '(', ')', '"']

Both `#` and `|` will be escaped, even though neither are a token on their own.

In [83]:
print(lisp_string(Symbol("A symbol with whitespace \"quotes\" and #| yikes.")))

A\ symbol\ with\ whitespace\ \"quotes\"\ and\ \#\|\ yikes.


In [84]:
lisp_string_table[EmptyList] = constantly('()')

In [85]:
print(lisp_string(empty_list()))

()


In [86]:
lisp_string_table[Pair] = lambda pair: lisp_list_to_lisp_string(pair)

Pairs will usually be rendered as lists. But sometimes, depending on the first expression in the list, it can be rendered differently.

For example, later we will have special syntax `(quote expression)` which can be read and printed as `'expression`.
Because of this, we'll have another dispatch table: `lisp_string_list_table`.

In [87]:
def lisp_list_to_lisp_string(pair):
    # we attempt a dispatch to the lisp_string_list_table
    if pair.a in lisp_string_list_table:
        return lisp_string_list_table[pair.a](pair)
    
    # otherwise, print the pair as a list.
    result = '('
    # Loop over each pair in the list.
    object = pair
    while True:
        t = type(object)
        # 1+ elements in the list.
        if   t is Pair:
            if object is not pair: result += ' '
            # object.a is assumed to be the next element in the list
            # object.b is assumed to be the rest of the list.
            result += lisp_string(object.a)
            object = object.b # object = rest of list
            
        # End of list
        elif t is EmptyList: return result + ')'
        # End of dotted list
        else: return result + ' . ' + lisp_string(object) + ')'

In [88]:
lisp_string_list_table = {}

In [89]:
def read_from_string(string): return read(StringStream(string))

In [90]:
print(lisp_string(read_from_string('(a 3.14159 (3+4j "the \\"quick\\" brown fox" . elephant) a\\ symbol h ())')))

(a 3.14159 (3+4j "the \"quick\" brown fox" . elephant) a\ symbol h ())


In [91]:
def lisp_eval(expression, environment):
    # Evaluation is in a loop to enable tail call elimination.
    while True:
        # If an expression is not already analyzed, macroexpand it and analyze it.
        if not type(expression) is Analyzed: expression = analyze(simplify(expand(expression, environment)))
        
        # Perform the operation by giving the analyzed_expression the current environment.
        result = expression(environment)
        # Tail call elimination handled here:
        if type(result) is TailCallEvaluationResult:
            # by looping instead of calling lisp_eval, we eliminate the need for a stack frame.
            (expression, environment) = (result.expression, result.environment)
        # Otherwise, no tail call, and we return the result.
        else: return result

In [92]:
simplify = lambda expression: expression

In [93]:
def expand(expression, environment):
    expression_type = type(expression)
    if expression_type is Pair:
        (first, rest) = (expression.a, expression.b)
        macro_procedure = environment_macro_procedure(environment, first)
        # Macro expansion?: Apply the macro procedure and expand the result.
        if macro_procedure:
            return expand(apply(macro_procedure, rest), environment)
        # Special form?: Dispatch to the expand_table
        elif first in expand_table:
            return expand_table[first](rest, environment)
        # Procedure application? Expand the procedure and each argument.
        else:
            (procedure, arguments) = parse_procedure_application(expression)
            results = [expand(procedure, environment)]
            while type(arguments) is Pair:
                results.append(expand(arguments.a, environment))
                arguments = arguments.b
            return python_list_to_lisp_list(results)

    # Symbol or self-evaluating? No expansion... but symbol macros _are_ a thing.
    else: return expression

In [94]:
expand_table = {}

A `TailCallEvaluationResult` is just an expression and an environment.

In [95]:
class TailCallEvaluationResult:
    def __init__(self, expression, environment):
        self.expression = expression
        self.environment = environment

In [96]:
def analyze(expression):
    expression_type = type(expression)
    # if an expression is a non-empty list it's a special form, a macro expansion, or a procedure application
    if expression_type is Pair:
        first = expression.a
        rest = expression.b
        # Special form?: Dispatch to the analyze_table
        if first in analyze_table: return analyze_table[first](rest)
        # Procedure Application? Analyze procedure application
        return analyze_procedure_application(expression)
    # Symbol?
    elif expression_type is Symbol: return analyze_symbol(expression)
    # Otherwise, we assume the expression is self-evaluating
    else: return analyze_self_evaluating(expression)

In [97]:
analyze_table = {}

In [98]:
def analyze_self_evaluating(expression):
    # Return the expression no matter what the evaluation environment is.
    def evaluate_self_evaluating(_evaluation_environment): return expression
    return Analyzed(evaluate_self_evaluating, expression)

In [99]:
class Analyzed:
    def __init__(self, eval, expression): 
        self.eval = eval
        self.expression = expression
    def __call__(self, environment): return self.eval(environment)

In [100]:
global_environment = None

In [101]:
def read_eval_print(string):
    result = lisp_eval(read_from_string(string), global_environment)
    print(lisp_string(result))
    return result

In [102]:
read_eval_print('3.14159')

3.14159


3.14159

In [103]:
read_eval_print('"Not very exciting to evaluate"')

"Not very exciting to evaluate"


'Not very exciting to evaluate'

In [104]:
def analyze_symbol(symbol):
    # Lookup the value in the evaluation_environment
    def evaluate_symbol(evaluation_environment):
        return environment_symbol_value(evaluation_environment, symbol)
    # If the symbol is a constant, look it up in the global environment ahead of time.
    if is_locked_lisp_symbol(symbol):
        value = environment_symbol_value(global_environment, symbol)
        return Analyzed(lambda _: value, symbol)
    else: return Analyzed(evaluate_symbol, symbol)

In [105]:
def is_locked_lisp_symbol(symbol): return symbol in locked_lisp_symbols

In [106]:
locked_lisp_symbols = []

In [107]:
expand_table[intern_symbol('define')] = lambda rest_expression, analysis_environment: expand_define(rest_expression, analysis_environment)

In [108]:
def expand_define(expression, environment):
    (name_symbol, value_expression) = parse_define(expression)
    return python_list_to_lisp_list([intern_symbol('define'), name_symbol, expand(value_expression, environment)])

In [109]:
analyze_table[intern_symbol('define')] = lambda rest_expression: analyze_define(rest_expression)

In [110]:
def parse_define(expression):
    if not type(expression) is Pair: error("Malformed define: missing name and value-expression. Expected (define symbol expression)")
    name_symbol = expression.a
    if not type (name_symbol) is Symbol: error("Malformed define: name should be a symbol. Expected (define symbol expression)")
    
    expression = expression.b
    if not type(expression) is Pair: error("Malformed define: missing value-expression. Expected (define symbol expression)")
    value_expression = expression.a
    
    if not type(expression.b) is EmptyList: error("Malformed define: too many expressions. Expected (define symbol expression)")
    return (name_symbol, value_expression)

In [111]:
def test_parse(parser, string):
    expression = read_from_string(string)
    try: print(str(parser(expression.b)))
    except Exception as e: print(str(e))

In [112]:
def test_parse_define(string): return test_parse(parse_define, string)

In [113]:
test_parse_define('(define a (+ 3 4))')
test_parse_define('(define "not a symbol" expression)')
test_parse_define('(define name)')
test_parse_define('(define name . expression)')
test_parse_define('(define name e1 e2)')

(Symbol("a"), Pair(Symbol("+"), Pair(3, Pair(4, EmptyList()))))
Malformed define: name should be a symbol. Expected (define symbol expression)
Malformed define: missing value-expression. Expected (define symbol expression)
Malformed define: missing value-expression. Expected (define symbol expression)
Malformed define: too many expressions. Expected (define symbol expression)


In [114]:
def analyze_define(expression):
    (name_symbol, value_expression) = parse_define(expression)
    if is_locked_lisp_symbol(name_symbol): error("Attempt to define locked symbol: " + lisp_string(name_symbol))
    value_expression = analyze(value_expression)
    
    # Evaluate the value expression, and bind it to name in the evaluation environment
    def evaluate_define(environment):
        value = lisp_eval(value_expression, environment)
        # TODO: verify that the symbol is not already bound in the environment
        environment_define_symbol_value(environment, name_symbol, value)
        # Return the name. Other reasonable choices would be to return value, or None.
        return name_symbol
    return Analyzed(evaluate_define, expression)

In [115]:
class Environment:
    def __init__(self, parent):
        self.bindings = {}
        self.parent = parent
        
    def __repr__(self):
        environments = []
        env = self
        while env:
            bindings = {}
            for (key, value) in env.bindings.items():
                bindings[key.name] = value
            environments.append(bindings)
            env = env.parent
        return '<Environment ' + str(environments) + '>'

In [116]:
def environment_frame_with_symbol_value(environment, symbol):
    # look for symbol in this environment frame, or the parent frame if not found.
    # Return the associated value.
    if not Environment or symbol in environment.bindings: return environment
    elif environment.parent: return environment_frame_with_symbol_value(environment.parent, symbol)

In [117]:
def environment_symbol_value(environment, symbol):
    environment_frame = environment_frame_with_symbol_value(environment, symbol)
    if environment_frame: return environment_frame.bindings[symbol]
    else: error("Symbol " + lisp_string(symbol) + " is unbound in the given environment")

In [118]:
def environment_set_symbol_value(environment, symbol, value):
    if not type(symbol) is Symbol: error("Got non-symbol object: " + lisp_string(symbol))
    environment_frame = environment_frame_with_symbol_value(environment, symbol)
    if environment_frame: environment_frame.bindings[symbol] = value
    else: error("Symbol " + lisp_string(symbol) + " is unbound in the given environment")

In [119]:
def environment_define_symbol_value(environment, symbol, value):
    # Add symbol to this environment frame, binding it to value.
    if not type(symbol) is Symbol: error("Got non-symbol object: " + lisp_string(symbol))
    environment.bindings[symbol] = value

In [120]:
def environment_macro_procedure(environment, symbol):
    try: 
        value = environment_symbol_value(environment, symbol)
        if type(value) is MacroProcedure: return value
        return None
    except Exception: return None

In [121]:
def environment_extend(environment): return Environment(environment)

In [122]:
test_environment = Environment(None)
environment_define_symbol_value(test_environment, intern_symbol('parent'), 'parent_top_level')
print(environment_symbol_value(test_environment, intern_symbol('parent')))

test_extended_environment = environment_extend(test_environment)
environment_define_symbol_value(test_extended_environment, intern_symbol('child'), 'child_in_extended')
environment_define_symbol_value(test_extended_environment, intern_symbol('parent'), 'parent_in_extended')

print(environment_symbol_value(test_extended_environment, intern_symbol('child')))
print(environment_symbol_value(test_extended_environment, intern_symbol('parent')))
print(environment_symbol_value(test_environment, intern_symbol('parent')))

environment_set_symbol_value(test_extended_environment, intern_symbol('child'), 'child_in_extended updated')
environment_set_symbol_value(test_extended_environment, intern_symbol('parent'), 'parent_in_extended updated')

print(environment_symbol_value(test_extended_environment, intern_symbol('child')))
print(environment_symbol_value(test_extended_environment, intern_symbol('parent')))
print(environment_symbol_value(test_environment, intern_symbol('parent')))

parent_top_level
child_in_extended
parent_in_extended
parent_top_level
child_in_extended updated
parent_in_extended updated
parent_top_level


In [123]:
try: environment_symbol_value(test_environment, intern_symbol('not-present'))
except Exception as e: print(str(e))

Symbol not-present is unbound in the given environment


In [124]:
global_environment = Environment(None)

In [125]:
read_eval_print('(define test-variable 3)')

test-variable


Symbol("test-variable")

In [126]:
read_eval_print('test-variable')

3


3

In [127]:
expand_table[intern_symbol('set!')] = lambda rest_expression, analysis_environment: expand_set(rest_expression, analysis_environment)

In [128]:
def expand_set(expression, environment):
    (name_symbol, value_expression) = parse_set(expression)
    return python_list_to_lisp_list([intern_symbol('set!'), name_symbol, expand(value_expression, environment)])

In [129]:
analyze_table[intern_symbol('set!')] = lambda rest_expression: analyze_set(rest_expression)

In [130]:
def analyze_set(expression):
    (name_symbol, value_expression) = parse_set(expression)
    if is_locked_lisp_symbol(name_symbol): error("Attempt to set! the value of locked symbol: " + lisp_string(name_symbol))
    value_expression = analyze(value_expression)
    
    # Evaluate the value expression, and bind it to name in the evaluation environment
    def evaluate_set(environment):
        value = lisp_eval(value_expression, environment)
        environment_set_symbol_value(environment, name_symbol, value)
        return value
    return Analyzed(evaluate_set, expression)

In [131]:
def parse_set(expression):
    if not type(expression) is Pair: error("Malformed set!: missing name and value-expression. Expected (set! symbol expression)")
    name_symbol = expression.a
    if not type (name_symbol) is Symbol: error("Malformed set!: name should be a symbol. Expected (set! symbol expression)")
    
    expression = expression.b
    if not type(expression) is Pair: error("Malformed set!: missing value-expression. Expected (set! symbol expression)")
    value_expression = expression.a
    
    if not type(expression.b) is EmptyList: error("Malformed set!: too many expressions. Expected (set! symbol expression)")
    return (name_symbol, value_expression)

In [132]:
def test_parse_set(string): return test_parse(parse_set, string)

In [133]:
test_parse_set('(set! a (+ 3 4))')
test_parse_set('(set! "not a symbol" expression)')
test_parse_set('(set! name)')
test_parse_set('(set! name . expression)')
test_parse_set('(set! name e1 e2)')

(Symbol("a"), Pair(Symbol("+"), Pair(3, Pair(4, EmptyList()))))
Malformed set!: name should be a symbol. Expected (set! symbol expression)
Malformed set!: missing value-expression. Expected (set! symbol expression)
Malformed set!: missing value-expression. Expected (set! symbol expression)
Malformed set!: too many expressions. Expected (set! symbol expression)


In [134]:
read_eval_print("""(define foo "foo")""")
read_eval_print("""foo""")
read_eval_print("""(set! foo "foo updated")""")
read_eval_print("""foo""")

foo
"foo"
"foo updated"
"foo updated"


'foo updated'

In [135]:
expand_table[intern_symbol('begin')] = lambda rest_expression, analysis_environment: expand_begin(rest_expression, analysis_environment)

In [136]:
def expand_begin(expression, environment):
    expressions = parse_begin(expression)
    result = [intern_symbol('begin')]
    for expression in expressions:
        result.append(expand(expression, environment))
    return python_list_to_lisp_list(result)

In [137]:
analyze_table[intern_symbol('begin')] = lambda rest_expression: analyze_begin(rest_expression)

In [138]:
def analyze_begin(expression):
    expressions = parse_begin(expression)
    analyzed_expressions = list(map(lambda expr: analyze(expr), expressions))
    
    def evaluate_begin(environment):
        # Evaluate all but the last expression
        for expr in analyzed_expressions[0:-1]:
            # discard the results
            lisp_eval(expr, environment)
        # Tail call elimination: return the last expression for lisp_eval to evaluate
        return TailCallEvaluationResult(analyzed_expressions[-1], environment)
    if len(analyzed_expressions) == 0: return None
    else: return Analyzed(evaluate_begin, expression)

In [139]:
def parse_begin(expression):
    expressions = []
    while type(expression) is Pair:
        expressions.append(expression.a)
        expression = expression.b
    if not type(expression) is EmptyList: error("Malformed begin: Expected expressions to be a proper list.")
    return expressions

In [140]:
def test_parse_begin(string): return test_parse(parse_begin, string)

In [141]:
test_parse_begin('(begin (define a 1) (define b 2) c)')
test_parse_begin('(begin)')
test_parse_begin('(begin . a)')
test_parse_begin('(begin a b . c)')

[Pair(Symbol("define"), Pair(Symbol("a"), Pair(1, EmptyList()))), Pair(Symbol("define"), Pair(Symbol("b"), Pair(2, EmptyList()))), Symbol("c")]
[]
Malformed begin: Expected expressions to be a proper list.
Malformed begin: Expected expressions to be a proper list.


In [142]:
read_eval_print('(begin (define test-a 1) (define test-b 2) (define test-a "one") test-a)')

"one"


'one'

In [143]:
read_eval_print('test-b')

2


2

In [144]:
expand_table[intern_symbol('if')] = lambda rest_expression, analysis_environment: expand_if(rest_expression, analysis_environment)

In [145]:
def expand_if(expression, environment):
    (test_expression, consequent_expression, alternative_expression) = parse_if(expression)
    return python_list_to_lisp_list([intern_symbol('if'),
                                        expand(test_expression, environment),
                                        expand(consequent_expression, environment),
                                        expand(alternative_expression, environment)])    

In [146]:
analyze_table[intern_symbol('if')] = lambda rest_expression: analyze_if(rest_expression)

In [147]:
def analyze_if(expression):
    (test_expression, consequent_expression, alternative_expression) = parse_if(expression)
    test_expression = analyze(test_expression)
    consequent_expression = analyze(consequent_expression)
    alternative_expression = analyze(alternative_expression)
    
    def evaluate_if(environment):
        # Tail call elimination: either return the consequent or alternative expression to lisp_eval.
        if lisp_eval(test_expression, environment):
            return TailCallEvaluationResult(consequent_expression, environment)
        else:
            return TailCallEvaluationResult(alternative_expression, environment)
    return Analyzed(evaluate_if, expression)

In [148]:
def parse_if(expression):
    if not type(expression) is Pair: error("Malformed if: missing test, consequent, and alternative")
    test_expression = expression.a
    
    expression = expression.b
    if not type(expression) is Pair: error("Malformed if: missing consequent and alternative")
    consequent_expression = expression.a
    
    expression = expression.b
    if not type(expression) is Pair: error("Malformed if: missing alternative")
    alternative_expression = expression.a
    
    if not type(expression.b) is EmptyList: error("Malformed if: too many expressions")
    return (test_expression, consequent_expression, alternative_expression)

In [149]:
def test_parse_if(string): return test_parse(parse_if, string)

In [150]:
test_parse_if('(if test consequent alternative)')
test_parse_if('(if)')
test_parse_if('(if test)')
test_parse_if('(if test consequent)')
test_parse_if('(if test consequent alternative what?)')

(Symbol("test"), Symbol("consequent"), Symbol("alternative"))
Malformed if: missing test, consequent, and alternative
Malformed if: missing consequent and alternative
Malformed if: missing alternative
Malformed if: too many expressions


In [151]:
def define_primitive(name_string, value):
    environment_define_symbol_value(global_environment, intern_symbol(name_string), value)

In [152]:
def define_constant(name_string, value):
    define_primitive(name_string, value)
    name_symbol = intern_symbol(name_string)
    if not name_symbol in locked_lisp_symbols:
        locked_lisp_symbols.append(name_symbol)

In [153]:
define_constant('#f', False)
define_constant('#t', True)
lisp_string_table[bool] = lambda boolean: '#t' if boolean else '#f'

In [154]:
try: read_eval_print('(define #t #f)')
except Exception as e: print(str(e))

Attempt to define locked symbol: \#t


In [155]:
read_eval_print('(begin (if #t (define a "true") (define a "false")) a)')
read_eval_print('(begin (if #f (define a "true") (define a "false")) a)')
None

"true"
"false"


In [156]:
expand_table[intern_symbol('lambda')] = lambda rest_expression, analysis_environment: expand_lambda(rest_expression, analysis_environment)

In [157]:
def expand_lambda(expression, environment):
    (parameter, body_expression) = parse_lambda(expression)
    return python_list_to_lisp_list([intern_symbol('lambda'), parameter, expand(body_expression, environment)])

In [158]:
analyze_table[intern_symbol('lambda')] = lambda rest_expression: analyze_lambda(rest_expression)

In [159]:
def analyze_lambda(expression):
    (parameter, body_expression) = parse_lambda(expression)
    body_expression = analyze(body_expression)
    
    def evaluate_lambda(environment):
        return Procedure(parameter, body_expression, environment)
    return Analyzed(evaluate_lambda, expression)

In [160]:
def parse_lambda(expression):
    if not type(expression) is Pair: error("Malformed lambda: missing parameter and body expression. Expected (lambda parameter expression)")
    
    parameter = expression.a
    if not type(parameter) is Symbol: error("Malformed lambda: Expected parameter to be a symbol.")
    
    expression = expression.b
    if not type(expression) is Pair: error("Malformed lambda: missing body expression. Expected (lambda parameter expression)")
    body_expression = expression.a
    
    expression = expression.b
    if not type(expression) is EmptyList: error("Malformed lambda: too many expressions. Expected (lambda parameter expression)")
    return (parameter, body_expression)

In [161]:
def test_parse_lambda(string): return test_parse(parse_lambda, string)

In [162]:
test_parse_lambda('(lambda parameter expression)')
test_parse_lambda('(lambda (a b c) expression)')
test_parse_lambda('(lambda parameter expression1 expression1)')
test_parse_lambda('(lambda)')
test_parse_lambda('(lambda parameter)')

(Symbol("parameter"), Symbol("expression"))
Malformed lambda: Expected parameter to be a symbol.
Malformed lambda: too many expressions. Expected (lambda parameter expression)
Malformed lambda: missing parameter and body expression. Expected (lambda parameter expression)
Malformed lambda: missing body expression. Expected (lambda parameter expression)


In [163]:
class Procedure:
    def __init__(self, parameter, expression, environment):
        self.parameter = parameter
        self.expression = expression
        self.environment = environment

In [164]:
def procedure_to_lisp_string(procedure):
    return '<lambda ' + lisp_string(procedure.parameter) + ' ' + lisp_string(procedure.expression) + '>'

lisp_string_table[Procedure] = procedure_to_lisp_string

In [165]:
read_eval_print('(lambda parameter a)')

<lambda parameter <__main__.Analyzed object at 0x0000029867A12EE0>>


In [166]:
expand_table[intern_symbol('macro-lambda')] = lambda rest_expression, environment: expand_macro_lambda(rest_expression, environment)

In [167]:
def expand_macro_lambda(expression, environment):
    (parameter, body_expression) = parse_lambda(expression)
    return python_list_to_lisp_list([intern_symbol('macro-lambda'), parameter, expand(body_expression, environment)])

In [168]:
analyze_table[intern_symbol('macro-lambda')] = lambda rest_expression: analyze_macro_lambda(rest_expression)

In [169]:
def analyze_macro_lambda(expression):
    (parameter, body_expression) = parse_lambda(expression)
    body_expression = analyze(body_expression)
    
    def evaluate_lambda(environment):
        return MacroProcedure(parameter, body_expression, environment)
    return Analyzed(evaluate_lambda, expression)

In [170]:
class MacroProcedure(Procedure): pass

In [171]:
def analyze_procedure_application(expression):
    (procedure_expression, argument_expression_list) = parse_procedure_application(expression)
    procedure_expression = analyze(procedure_expression)
    
    analyzed_argument_expression_list = empty_list()
    expr = argument_expression_list
    while type(expr) is Pair:
        analyzed_argument_expression_list = Pair(analyze(expr.a), analyzed_argument_expression_list)
        expr = expr.b
    
    def evaluate_procedure_application(environment):
        # Evaluate procedure and all arguments
        procedure = lisp_eval(procedure_expression, environment)
        
        argument_list = empty_list()
        arguments = []
        expr = analyzed_argument_expression_list
        while type(expr) is Pair:
            argument = lisp_eval(expr.a, environment)
            argument_list = Pair(argument, argument_list)
            arguments.append(argument)
            expr = expr.b
        
        # Tail call elimination. Return the procedure expression and environment back to lisp_eval.
        if type(procedure) is Procedure:
            # Extend the procedure environment to have parameter bound to to the passed in argument_list
            environment = environment_extend(procedure.environment)
            environment_define_symbol_value(environment, procedure.parameter, argument_list)
            return TailCallEvaluationResult(procedure.expression, environment)
        # procedure is a python function.
        else:
            arguments.reverse()
            return procedure(*arguments)
    return Analyzed(evaluate_procedure_application, expression)

In [172]:
def parse_procedure_application(expression):
    whole_expression = expression
    argument_expressions = []
    procedure_expression = expression.a
    argument_expressions = expression = expression.b
    while type(expression) is Pair: expression = expression.b
    if type(expression) is not EmptyList: error("Procedure application expects a proper list: " + lisp_string(whole_expression))
    return (procedure_expression, argument_expressions)

In [173]:
expand_table[intern_symbol('quote')] = lambda rest_expression, environment: expand_quote(rest_expression, environment)

In [174]:
def expand_quote(expression, environment):
    return Pair(intern_symbol('quote'), expression)

In [175]:
analyze_table[intern_symbol('quote')] = lambda rest_expression: analyze_quote(rest_expression)

In [176]:
def analyze_quote(expression):
    quoted_expression = parse_quote(expression)
    def evaluate_quote(environment):
        return quoted_expression
    return Analyzed(evaluate_quote, expression)

In [177]:
def parse_quote(expression):
    if not type(expression) is Pair: error("Malformed quote: expected expression")
    quoted_expression = expression.a
    if not type(expression.b) is EmptyList: error("Malformed quote: expected exactly one expression.")
    return quoted_expression

In [178]:
def test_parse_quote(string): return test_parse(parse_quote, string)

In [179]:
test_parse_quote('(quote expression)')
test_parse_quote('(quote)')
test_parse_quote('(quote . rest)')
test_parse_quote('(quote a b)')

Symbol("expression")
Malformed quote: expected expression
Malformed quote: expected expression
Malformed quote: expected exactly one expression.


In [180]:
read_eval_print('(quote (the quick brown (fox)))')

(the quick brown (fox))


Pair(Symbol("the"), Pair(Symbol("quick"), Pair(Symbol("brown"), Pair(Pair(Symbol("fox"), EmptyList()), EmptyList()))))

In [181]:
register_lisp_delimiting_token("'")
lisp_token_table["'"] = lambda stream: Pair(intern_symbol('quote'), Pair(read(stream), empty_list()))
lisp_string_list_table[intern_symbol('quote')] = lambda pair: "'" + lisp_string(pair.b.a)

In [182]:
read_eval_print("'(the quick brown (fox))")

(the quick brown (fox))


Pair(Symbol("the"), Pair(Symbol("quick"), Pair(Symbol("brown"), Pair(Pair(Symbol("fox"), EmptyList()), EmptyList()))))

In [183]:
read_eval_print("''(the quick brown '(fox))")

'(the quick brown '(fox))


Pair(Symbol("quote"), Pair(Pair(Symbol("the"), Pair(Symbol("quick"), Pair(Symbol("brown"), Pair(Pair(Symbol("quote"), Pair(Pair(Symbol("fox"), EmptyList()), EmptyList())), EmptyList())))), EmptyList()))

In [184]:
define_primitive('eval-python', eval)
define_primitive('exec-python', exec)
define_primitive('procedure->python', lambda procedure: lisp_procedure_to_python(procedure))
define_primitive('python-list', lambda *arguments: [*arguments])

In [185]:
# TODO: bootstrap
read_eval_print("""
(begin
    (exec-python "global op; import operator as op")
    (define attr (eval-python "op.attrgetter"))
    (define eq? (eval-python "op.is_"))
    
    (define empty?
        (lambda args 
            (begin 
                (define object (first args))
                (eq? object ()))))
    
    (define type? (eval-python "isinstance"))
    (define type-predicate 
        (lambda args
            (begin
                (define type (first args)) 
                (lambda args
                    (begin
                        (define object (first args))
                        (type? object type))))))
    
    (define pair (eval-python "Pair"))
    (define first (attr "a"))
    (define rest (attr "b"))
    
    (define pair? (type-predicate pair))
    (define cons pair)
    (define car first)
    (define cdr rest)
    
    (define integer (eval-python "int"))
    (define integer? (type-predicate integer))
    
    (define complex (eval-python "complex"))
    (define complex? (type-predicate complex))
    
    (define boolean (eval-python "bool"))
    (define boolean? (type-predicate boolean))
    
    (define floating-point (eval-python "float"))
    (define floating-point? (type-predicate floating-point))

    (define procedure (eval-python "Procedure"))
    (define procedure? (type-predicate procedure))

    (define symbol (eval-python "Symbol"))
    (define symbol? (type-predicate symbol)))
""")

symbol?


Symbol("symbol?")

In [186]:
read_eval_print("""(begin
(define foo
    (lambda args
        (begin
            (define a (first args))
            (define true (first (rest args)))
            (define false (first (rest (rest args))))
            (if a (true a) (false a)))))
(foo "a" (lambda _ (begin (define b "true") b)) (lambda _ "false")))""")

"true"


'true'

In [187]:
read_eval_print("""((lambda args (begin (define x (first args)) x)) "a" "b" "c")""")
read_eval_print("""(begin
(define f
    (lambda args
        (begin
            (define a (first args))
            (define b (first (rest args)))
            (define c (first (rest (rest args))))
            (if a b  c))))
(define b (f #t "b" "c"))
(define c (f #f "b" "c")))""")
read_eval_print('b')
read_eval_print('c')

"a"
c
"b"
"c"


'c'

In [188]:
def lisp_procedure_to_python(procedure):
    if isinstance(procedure, Procedure):
        def function(*arguments):
            environment = environment_extend(procedure.environment)
            environment_define_symbol_value(environment, procedure.parameter, python_list_to_lisp_list(arguments))
            return lisp_eval(procedure.expression, environment)
        return function
    else: return procedure

In [189]:
# TODO: bootstrap
read_eval_print("""
(begin
  (exec-python "global op; import operator as op")
  (define + (eval-python "op.add"))
  (define - (eval-python "op.sub"))
  (define * (eval-python "op.mul"))
  (define / (eval-python "op.truediv"))
  
  (define > (eval-python "op.gt"))
  (define < (eval-python "op.lt"))
  (define >= (eval-python "op.ge"))
  (define <= (eval-python "op.le"))
  (define = (eval-python "op.eq"))
  
  (define not (eval-python "op.not_")))
""")

not


Symbol("not")

In [190]:
read_eval_print('(+ 1 2)')

3


3

In [191]:
read_eval_print("""
(define append 
    (lambda args
        (begin
            (define a (first args))
            (define b (first (rest args)))
            (if (empty? a) 
                b
                (cons (first a) (append (rest a) b))))))
""")

append


Symbol("append")

In [192]:
read_eval_print("(append '(one two three) '(4 5 6))")

(one two three 4 5 6)


Pair(Symbol("one"), Pair(Symbol("two"), Pair(Symbol("three"), Pair(4, Pair(5, Pair(6, EmptyList()))))))

In [193]:
read_eval_print("""
(begin
    (exec-python "global math; import math")
    (define cos (eval-python "math.cos"))
    (define sin (eval-python "math.sin")))
""")

sin


Symbol("sin")

In [194]:
read_eval_print("""(begin
(define square (lambda args (begin (define x (first args)) (* x x))))

(define floating-point=
    (lambda args
        (begin
            (define v1 (first args))
            (define v2 (first (rest args)))
            (define epsilon (first (rest (rest args))))
            (if (> v1 (- v2 epsilon))
                (if (< v1 (+ v2 epsilon))
                    #t
                    #f)
                #f))))

(define cos-squared+sin-squared
    (lambda args 
        (begin 
            (define x (first args))
            (+ (square (cos x)) (square (sin x))))))
(floating-point= 1.0 (cos-squared+sin-squared .4) .01))
""")
read_eval_print("""(floating-point= 1.0 (cos-squared+sin-squared 42.424242) .01)""")

#t
#t


True

In [195]:
test_lisp_append = read_eval_print('(procedure->python append)')
test_list_a = read_eval_print("'(a b c)")
test_list_b = read_eval_print("'(d e f)")
lisp_string(test_lisp_append(test_list_a, test_list_b))

<function lisp_procedure_to_python.<locals>.function at 0x0000029867A6C940>
(a b c)
(d e f)


'(a b c d e f)'

In [196]:
read_eval_print('(python-list 1 "two" 3.0 4+5j)')

[1, 'two', 3.0, (4+5j)]


[1, 'two', 3.0, (4+5j)]

In [197]:
read_eval_print('(define list (lambda args args))')

list


Symbol("list")

In [198]:
read_eval_print("""(list 1 2 3)""")

(1 2 3)


Pair(1, Pair(2, Pair(3, EmptyList())))

In [199]:
def apply(procedure, argument_list):
    if isinstance(procedure, Procedure):
        environment = environment_extend(procedure.environment)
        environment_define_symbol_value(procedure.environment, procedure.parameter, argument_list)
        return lisp_eval(procedure.expression, environment)
    else:
        return procedure(*lisp_list_to_python_list(argument_list))

In [200]:
def lisp_list_to_python_list(expr):
    arguments = []
    while type(expr) is Pair:
        arguments.append(expr.a)
        expr = expr.b
    return arguments

In [201]:
read_eval_print("""(define apply (eval-python "apply"))""")

apply


Symbol("apply")

In [202]:
# TODO: boostrap
read_eval_print("""(begin
(define second (lambda args (first (apply rest args))))
(define third (lambda args (first (rest (apply rest args)))))
(define fourth (lambda args (first (rest (rest (apply rest args)))))))
""")

fourth


Symbol("fourth")

In [203]:
read_eval_print("""(second (list 1 2 3))""")
read_eval_print("""(third (list 1 2 3))""")

2
3


3

In [204]:
# TODO: boostrap
read_eval_print("""
(define positional-spec->definition
    (lambda args
        (begin
            (define parameter (first args))
            (define arguments (first (rest args)))
            (list 'if (list 'pair? arguments) 
                      (list 'define parameter (list 'pop! arguments))
                      (list 'missing-positional-argument-error (list 'quote parameter))))))""")

positional-spec->definition


Symbol("positional-spec->definition")

In [205]:
read_eval_print("""(positional-spec->definition 'a '<argument>)""")
None

(if (pair? <argument>) (define a (pop! <argument>)) (missing-positional-argument-error 'a))


In [206]:
# TODO: boostrap
read_eval_print("""
(define optional-spec->definition
    (lambda args
        (begin
            (define spec (first args))
            (define spec-name (first spec))
            (define spec-default (second spec))
            (define arguments (first (rest args)))
            (list 'if (list 'pair? arguments) 
                      (list 'define spec-name (list 'pop! arguments))
                      (list 'define spec-name spec-default)))))""")

optional-spec->definition


Symbol("optional-spec->definition")

In [207]:
# TODO: boostrap
read_eval_print("""(optional-spec->definition '(optional-c 'default-value) '<argument>)""")
None

(if (pair? <argument>) (define optional-c (pop! <argument>)) (define optional-c 'default-value))


In [208]:
# TODO: boostrap
read_eval_print("""
(define rest-spec->definition
    (lambda args
        (begin
            (define spec (first args))
            (define arguments (first (rest args)))
            (list 'define spec arguments))))""")

rest-spec->definition


Symbol("rest-spec->definition")

In [209]:
read_eval_print("""(rest-spec->definition 'rest '<argument>)""")
None

(define rest <argument>)


In [210]:
# TODO: boostrap
read_eval_print("""
(define parameter-spec->definition
    (lambda args
        (begin
            (define spec (first args))
            (define arguments (first (rest args)))
            (if (pair? spec)
                (optional-spec->definition spec arguments)
                (positional-spec->definition spec arguments)))))""")

parameter-spec->definition


Symbol("parameter-spec->definition")

In [211]:
read_eval_print("""(parameter-spec->definition 'a '<argument>)""")
None

(if (pair? <argument>) (define a (pop! <argument>)) (missing-positional-argument-error 'a))


In [212]:
read_eval_print("""(parameter-spec->definition '(optional-c 'default-c) '<argument>)""")
None

(if (pair? <argument>) (define optional-c (pop! <argument>)) (define optional-c 'default-c))


In [213]:
# TODO: boostrap
read_eval_print("""
(define parameter-specs->definitions
    (lambda args
        (begin
            (define parameters (first args))
            (define arguments (first (rest args)))
            (if (pair? parameters) 
                (cons (parameter-spec->definition (first parameters) arguments)
                      (parameter-specs->definitions (rest parameters) arguments))
                (if (empty? parameters)
                    ()
                    (list (rest-spec->definition parameters arguments)))))))""")

parameter-specs->definitions


Symbol("parameter-specs->definitions")

In [214]:
# TODO: boostrap
read_eval_print("""
(define parameter-specs->definition
    (lambda args
        (begin
            (define parameters (first args))
            (define arguments (first (rest args)))
            (cons 'begin (parameter-specs->definitions parameters arguments)))))""")

parameter-specs->definition


Symbol("parameter-specs->definition")

In [215]:
read_eval_print("""(parameter-specs->definition '(a b (optional-c #f) . rest) 'argument)""")
None

(begin (if (pair? argument) (define a (pop! argument)) (missing-positional-argument-error 'a)) (if (pair? argument) (define b (pop! argument)) (missing-positional-argument-error 'b)) (if (pair? argument) (define optional-c (pop! argument)) (define optional-c \#f)) (define rest argument))


In [216]:
# TODO: boostrap
read_eval_print("""
(define fn
    (macro-lambda args
        (begin
            (define parameter-specs (first args))
            (define body (rest args))
            (define <arguments> (make-symbol "arguments"))
            (list 'lambda <arguments> (cons 'begin (cons (parameter-specs->definition parameter-specs <arguments>) body))))))""")
read_eval_print("""fn""")

fn


In [217]:
# TODO: boostrap
read_eval_print("""(define make-symbol (eval-python "Symbol"))""")

make-symbol


Symbol("make-symbol")

In [218]:
# TODO: boostrap
read_eval_print("""
(define pop!
    (macro-lambda args
        (begin
            (define stack (first args))
            (list 'apply 
                (list 'lambda 'value
                    (list 'begin 
                        (list 'set! stack (list 'rest stack))
                        'value))
                (list 'first stack)))))""")

pop!


Symbol("pop!")

In [219]:
read_eval_print("""(apply pop! '(stack))""")
None

(apply (lambda value (begin (set! stack (rest stack)) value)) (first stack))


In [220]:
lisp_string(expand(read_from_string("""(fn (a b (c #f) . rest) (list 'a a 'b b 'c c 'rest rest))"""), global_environment))

"(lambda arguments (begin (begin (if (pair? arguments) (define a (apply (lambda value (begin (set! arguments (rest arguments)) value)) (first arguments))) (missing-positional-argument-error 'a)) (if (pair? arguments) (define b (apply (lambda value (begin (set! arguments (rest arguments)) value)) (first arguments))) (missing-positional-argument-error 'b)) (if (pair? arguments) (define c (apply (lambda value (begin (set! arguments (rest arguments)) value)) (first arguments))) (define c \\#f)) (define rest arguments)) (list 'a a 'b b 'c c 'rest rest)))"

In [221]:
read_eval_print("""(define test-fn (fn (a b (c #f) . rest) (list 'a a 'b b 'c c 'rest rest)))""")
read_eval_print("""(test-fn 1 2)""")
read_eval_print("""(test-fn 1 2 3 4 5 6)""")
None

test-fn
(a 1 b 2 c #f rest ())
(a 1 b 2 c 3 rest (4 5 6))


In [222]:
read_eval_print("""(define missing-positional-argument-error (fn (argument-name) (error (+ "Argument missing: " (symbol-name argument-name)))))""")

missing-positional-argument-error


Symbol("missing-positional-argument-error")

In [223]:
# TODO: boostrap
read_eval_print("""(define symbol-name (eval-python "op.attrgetter('name')"))""")

symbol-name


Symbol("symbol-name")

In [224]:
# TODO: boostrap
read_eval_print("""
(define list*
    (fn (arg . args)
        (if (empty? args)
            arg
            (cons arg (apply list* args)))))
""")

list*


Symbol("list*")

In [225]:
read_eval_print("""(list* 1 2 3 (list 4 5 6))""")

(1 2 3 4 5 6)


Pair(1, Pair(2, Pair(3, Pair(4, Pair(5, Pair(6, EmptyList()))))))

In [226]:
# TODO: boostrap
read_eval_print("""
(define cond->if
    (fn (cond-specs)
        (if (pair? cond-specs)
            (cond-spec->if (first cond-specs) (rest cond-specs))
            #f)))
""")

cond->if


Symbol("cond->if")

In [227]:
# TODO: boostra
read_eval_print("""
(define cond-spec->if
    (fn (cond-spec cond-specs)
        (list 'if (first cond-spec)
                  (cons 'begin (rest cond-spec))
                  (cond->if cond-specs))))""")

cond-spec->if


Symbol("cond-spec->if")

In [228]:
read_eval_print("""
(cond->if
    '(((pair? a) (first a))
      ((number? a) (print a) (+ a 2))
      (#t "default case")))
""")
None

(if (pair? a) (begin (first a)) (if (number? a) (begin (print a) (+ a 2)) (if \#t (begin "default case") #f)))


In [229]:
# TODO: boostrap
read_eval_print("""(define cond (macro-lambda cond-specs (cond->if cond-specs)))""")

cond


Symbol("cond")

In [230]:
read_eval_print("""
(define test-cond
    (fn (object)
        (cond 
            ((pair? object) (list (first object) "It's a pair!"))
            ((integer? object) (list (+ object 1) "Behold! it's an integer"))
            ((complex? object) (list (* object 1j) "It is very complicated."))
            ((empty? object) (list object "The glass is 0% full."))
            (#t (list object "What is this garbage?"))))))
""")

test-cond


Symbol("test-cond")

In [231]:
read_eval_print("""(test-cond (cons 1 2))""")
read_eval_print("""(test-cond ())""")
read_eval_print("""(test-cond 3.14159)""")
read_eval_print("""(test-cond 3)""")
None

(1 "It's a pair!")
(() "The glass is 0% full.")
(3.14159 "What is this garbage?")
(4 "Behold! it's an integer")


In [232]:
# TODO: boostrap

read_eval_print("""
(define let->lambda
    (fn (bindings body)
        (list* (list* 'fn (map first bindings)
                    body)
              (map second bindings))))
""")

let->lambda


Symbol("let->lambda")

In [233]:
# TODO: boostrap

read_eval_print("""
(define map
    (fn (f list)
        (if (pair? list)
            (cons (f (first list)) (map f (rest list)))
            ())))""")

map


Symbol("map")

In [234]:
read_eval_print("""(map (fn (x) (+ x 1)) '(1 2 3))""")
None

(2 3 4)


In [235]:
read_eval_print("""(let->lambda '((a 1) (b 2) (c 'c)) '((define d 'd) (list a b c d)))""")
None

((fn (a b c) (define d 'd) (list a b c d)) 1 2 'c)


In [236]:
read_eval_print("""((fn (a b c) (define d 'd) (list a b c d)) 1 2 'c)""")
None

(1 2 c d)


In [237]:
read_eval_print("""(define let (macro-lambda arguments (let->lambda (first arguments) (rest arguments))))""")

let


Symbol("let")

In [238]:
read_eval_print("""
(let ((a 1)
      (b 2)
      (c 3))
  (define d 4)
  (list a b c d))""")
None

(1 2 3 4)


In [239]:
# TODO: boostrap

read_eval_print("""
(define reverse
    (let ((reverse-iter #f))
        (set! reverse-iter
            (fn (list result)
                (if (empty? list)
                    result
                    (reverse-iter (rest list) (cons (first list) result)))))
        (fn (list) (reverse-iter list ()))))
""")

reverse


Symbol("reverse")

In [240]:
# TODO: boostrap

read_eval_print("""(begin
(define map-reverse
    (let ((map-reverse-iter #f))
        (set! map-reverse-iter
            (fn (f list result)
                (if (pair? list)
                    (map-reverse-iter f (rest list) (cons (f (first list)) result))
                    result)))
        (fn (f list) (map-reverse-iter f list ()))))
(define map (fn (f list) (reverse (map-reverse f list)))))
""")

map


Symbol("map")

In [241]:
read_eval_print("""(map (fn (x) (+ x 1)) '(1 2 3))""")

(2 3 4)


Pair(2, Pair(3, Pair(4, EmptyList())))

In [242]:
# TODO: boostrap

read_eval_print("""
(define and->if
    (fn (arguments)
        (if (pair? arguments) 
            (if (empty? (rest arguments))
                (first arguments)
                (list 'if (first arguments) 
                          (and->if (rest arguments))
                          #f))
            #f)))
""")

and->if


Symbol("and->if")

In [243]:
read_eval_print("""(and->if '(a b c d))""") # (if a (and b c d) #f)
read_eval_print("""(and->if '())""") # #f
read_eval_print("""(and->if '(v))""") # v

(if a (if b (if c d #f) #f) #f)
#f
v


Symbol("v")

In [244]:
# TODO: boostrap

read_eval_print("""(define and (macro-lambda arguments (and->if arguments)))""")

and


Symbol("and")

In [245]:
read_eval_print("""
(define quasiquote->list
    (fn (arg)
        (cond
            ((pair? arg)
             (cond
                 ((eq? (first arg) 'quasiquote-unquote) (second arg))
                 ((and (pair? (first arg)) (eq? (first (first arg)) 'quasiquote-unquote-splice))
                  (list 'append (second (first arg)) (quasiquote->list (rest arg))))
                 (#t (list 'cons (quasiquote->list (first arg)) (quasiquote->list (rest arg))))))
            (#t (list 'quote arg)))))
""")

quasiquote->list


Symbol("quasiquote->list")

In [246]:
read_eval_print("""(quasiquote->list 'a)""") # 'a
read_eval_print("""(quasiquote->list '(quasiquote-unquote (list 'a)))""") # (a)
read_eval_print("""(quasiquote->list '(a b c (quasiquote-unquote-splice (list 1 2 3)) . ()))""") # (a b c 1 2 3)
None

'a
(list 'a)
(cons 'a (cons 'b (cons 'c (append (list 1 2 3) '()))))


In [247]:
read_eval_print("""(define quasiquote (macro-lambda args (quasiquote->list (first args))))""")

quasiquote


Symbol("quasiquote")

In [248]:
read_eval_print("""(quasiquote a)""") # a
read_eval_print("""(quasiquote (quasiquote-unquote (list 'a)))""") # (a)
read_eval_print("""(quasiquote (a b c (quasiquote-unquote-splice (list 1 2 3)) . ()))""") # (a b c 1 2 3)
None

a
(a)
(a b c 1 2 3)


In [249]:
register_lisp_delimiting_token("`")
lisp_token_table["`"] = lambda stream: tag_expression('quasiquote', read(stream))
register_lisp_delimiting_token(",")
lisp_token_table[","] = lambda stream: tag_expression('quasiquote-unquote', read(stream))
register_lisp_delimiting_token(",@")
lisp_token_table[",@"] = lambda stream: tag_expression('quasiquote-unquote-splice', read(stream))

In [250]:
def tag_expression(symbol_name, expression):
    return python_list_to_lisp_list([intern_symbol(symbol_name), expression])

In [251]:
read_eval_print(""" `a """) # 'a
read_eval_print(""" `,(list 'a) """) # (a)
read_eval_print(""" `(a b c ,@(list 1 2 3) . ()) """) # (a b c 1 2 3)

a
(a)
(a b c 1 2 3)


Pair(Symbol("a"), Pair(Symbol("b"), Pair(Symbol("c"), Pair(1, Pair(2, Pair(3, EmptyList()))))))

In [252]:
read_eval_print("""(define error (eval-python "error"))""")

error


Symbol("error")

In [253]:
read_eval_print("""(define quasiquote-unquote (lambda _ (error "quasiquote-unquote is invalid outside of a quasiquote form")))""") 
read_eval_print("""(define quasiquote-unquote-splice (lambda _ (error "quasiquote-unquote-splice is invalid outside of a quasiquote form")))""") 

quasiquote-unquote
quasiquote-unquote-splice


Symbol("quasiquote-unquote-splice")

In [254]:
try: read_eval_print(""" ,"hello" """)
except Exception as e: print(str(e))

quasiquote-unquote is invalid outside of a quasiquote form


In [255]:
# TODO: boostrap

read_eval_print("""
(define or->if
    (fn (arguments)
        (if (pair? arguments)
            (if (empty? (rest arguments))
                (first arguments)
                (let ((<value> (make-symbol "<value>")))
                    `(let ((,<value> ,(first arguments)))
                        (if ,<value> 
                            ,<value>
                            ,(or->if (rest arguments))))))
            #t)))
""")

or->if


Symbol("or->if")

In [256]:
read_eval_print("""(or->if '())""")
read_eval_print("""(or->if '(a))""")
read_eval_print("""(or->if '(a b c))""")
None

#t
a
(let ((<value> a)) (if <value> <value> (let ((<value> b)) (if <value> <value> c))))


In [257]:
# TODO: boostrap

read_eval_print("""(define or (macro-lambda args (or->if args)))""")

or


Symbol("or")

In [258]:
read_eval_print("""(or #f #f 2)""")

2


2

In [259]:
# TODO: boostrap

read_eval_print("""
(define let*->let
    (fn (bindings body)
        (cond
            ((pair? bindings) `(let (,(first bindings)) ,(let*->let (rest bindings) body)))
            (#t `(begin ,@body)))))
""")

let*->let


Symbol("let*->let")

In [260]:
read_eval_print("""(let*->let '((a 1) (b a) (c b)) '((list a b c)))""")
None

(let ((a 1)) (let ((b a)) (let ((c b)) (begin (list a b c)))))


In [261]:
# TODO: boostrap

read_eval_print("""
(define let* (macro-lambda arguments (let*->let (first arguments) (rest arguments))))
""")

let*


Symbol("let*")

In [262]:
read_eval_print("""
(let* ((a 1)
       (b (+ a 1))
       (c (* b 2)))
 (list 'a a 'b b 'c c))
""")
None

(a 1 b 2 c 4)


In [263]:
# TODO: boostrap

read_eval_print("""
(define letrec-form
    (fn (bindings body)
        `(let ,(map (fn (binding) `(,(first binding) #f)) bindings)
            (begin
              ,@(map (fn (binding) `(set! ,(first binding) ,(second binding))) bindings))
              ,@body)))
""")

letrec-form


Symbol("letrec-form")

In [264]:
read_eval_print("""(letrec-form '((a 1) (b (+ 2 a)) (c (* 3 b))) '((list a b c)))""")
None

(let ((a \#f) (b \#f) (c \#f)) (begin (set! a 1) (set! b (+ 2 a)) (set! c (* 3 b))) (list a b c))


In [265]:
# TODO: boostrap

read_eval_print("""(define letrec (macro-lambda arguments (letrec-form (first arguments) (rest arguments))))""")

letrec


Symbol("letrec")

In [266]:
test_expression = analyze(expand(read(StringStream("""
(letrec ((is-even? (fn (n)
                     (or (= 0 n) (is-odd? (- n 1)))))
         (is-odd? (fn (n)
                    (and (not (= 0 n))
                         (is-even? (- n 1))))))
    (is-odd? 10001))
""")), global_environment))
print(lisp_string(test_expression.expression))

((lambda arguments (begin (begin (if (pair? arguments) (define is-even? (apply (lambda value (begin (set! arguments (rest arguments)) value)) (first arguments))) (missing-positional-argument-error 'is-even?)) (if (pair? arguments) (define is-odd? (apply (lambda value (begin (set! arguments (rest arguments)) value)) (first arguments))) (missing-positional-argument-error 'is-odd?))) (begin (set! is-even? (lambda arguments (begin (begin (if (pair? arguments) (define n (apply (lambda value (begin (set! arguments (rest arguments)) value)) (first arguments))) (missing-positional-argument-error 'n))) ((lambda arguments (begin (begin (if (pair? arguments) (define <value> (apply (lambda value (begin (set! arguments (rest arguments)) value)) (first arguments))) (missing-positional-argument-error '<value>))) (if <value> <value> (is-odd? (- n 1))))) (= 0 n))))) (set! is-odd? (lambda arguments (begin (begin (if (pair? arguments) (define n (apply (lambda value (begin (set! arguments (rest arguments)

In [377]:
lisp_eval(test_expression, global_environment)

True

In [370]:
def lisp(string): return lisp_string(lisp_eval(python_list_to_lisp_list([intern_symbol('begin'), read_from_string(string)]), global_environment))

In [374]:
lisp("""
(define-simplify-rule '(begin) #f)
(define-simplify-rule '(begin (? a)) a)
;; Eliminate constant expressions within begin
(define-simplify-rule '(begin (?? before) (? c constant-expression?) (? a) (?? after))
                      `(begin ,@before ,a ,@after))
;; Lift inner begin forms to outer begin forms
(define-simplify-rule '(begin (?? before) (begin (?? inside)) (?? after)) 
                      `(begin ,@before ,@inside ,@after))

""")

Exception: Symbol define-simplify-rule is unbound in the given environment

In [270]:
read_eval_print("""
(define scheme:eval
    (fn (k expression (environment scheme:global-environment))
        (if (procedure? expression)
            (expresssion k environment)
            ((scheme:analyze (scheme:expand expression environment)) k environment))))
""")

scheme:eval


Symbol("scheme:eval")

In [271]:
# TODO: boostrap

read_eval_print("""(define make-environment-frame (eval-python "Environment"))""")
read_eval_print("""(define environment-symbol-value (eval-python "environment_symbol_value"))""")
read_eval_print("""(define environment-macro-procedure (eval-python "environment_macro_procedure"))""")
read_eval_print("""(define environment-define-symbol-value (eval-python "environment_define_symbol_value"))""")
read_eval_print("""(define environment-set-symbol-value (eval-python "environment_set_symbol_value"))""")
read_eval_print("""(define environment-extend (eval-python "environment_extend"))""")

make-environment-frame
environment-symbol-value
environment-macro-procedure
environment-define-symbol-value
environment-set-symbol-value
environment-extend


Symbol("environment-extend")

In [272]:
read_eval_print("""(define scheme:global-environment (make-environment-frame #f))""")

scheme:global-environment


Symbol("scheme:global-environment")

In [273]:
read_eval_print("""
(define scheme:expand
    (fn (expression environment)
        (cond
            ((pair? expression) (scheme:expand-combination expression environment))
            ;; Symbol
            ((symbol? expression) (scheme:expand-symbol expression environment))
            ;; Self-evaluating
            (#t expression))))
""")

scheme:expand


Symbol("scheme:expand")

In [274]:
read_eval_print("""
(define scheme:expand-combination
    (fn (expression environment)
        (let ((macro-procedure #f))
            (cond
                ;; special form?
                ((hash-contains-key? scheme:expand-table (first expression))
                 ((hash-ref scheme:expand-table (first expression)) (rest expression) environment))
                ;; macro-procedure application?
                ((set! macro-procedure (environment-macro-procedure environment (first expression)))
                 ;; Apply the macro procedure to the rest of the unevaluated arguments
                 ;; And then expand the result.
                 (scheme:apply (fn (expression) (scheme:expand expression environment))
                               macro-procedure (rest expression)))
                ;; procedure application
                (#t (scheme:expand-procedure-application expression environment))))))
""")

scheme:expand-combination


Symbol("scheme:expand-combination")

In [275]:
read_eval_print("""(define make-hash-table (eval-python "dict"))""")
read_eval_print("""(define hash-contains-key? (eval-python "op.contains"))""")
read_eval_print("""(define hash-ref (eval-python "op.getitem"))""")
read_eval_print("""(define hash-set! (eval-python "op.setitem"))""")

make-hash-table
hash-contains-key?
hash-ref
hash-set!


Symbol("hash-set!")

In [276]:
read_eval_print("""(define scheme:expand-table (make-hash-table))""")

scheme:expand-table


Symbol("scheme:expand-table")

In [277]:
read_eval_print("""(scheme:expand '1.2345 scheme:global-environment)""")

1.2345


1.2345

In [278]:
read_eval_print("""
(define scheme:expand-symbol (fn (expression environment) expression))
""")

scheme:expand-symbol


Symbol("scheme:expand-symbol")

In [279]:
read_eval_print("""(scheme:expand 'symbol scheme:global-environment)""")

symbol


Symbol("symbol")

In [280]:
read_eval_print("""
(define scheme:expand-procedure-application
    (fn (expression environment)
        (map (fn (expression) (scheme:expand expression environment)) expression)))
""")

scheme:expand-procedure-application


Symbol("scheme:expand-procedure-application")

In [281]:
read_eval_print("""(scheme:expand '(+ 1 a) scheme:global-environment)""")

(+ 1 a)


Pair(Symbol("+"), Pair(1, Pair(Symbol("a"), EmptyList())))

In [282]:
read_eval_print("""(hash-set! scheme:expand-table 'define (lambda args (apply expand-define args)))""")
read_eval_print("""
(define expand-define
    (fn (expression environment)
        `(define ,(first expression) ,(scheme:expand (second expression) environment))))
""")

None
expand-define


Symbol("expand-define")

In [283]:
read_eval_print("""(scheme:expand '(define a (+ 1 2)) scheme:global-environment)""")

(define a (+ 1 2))


Pair(Symbol("define"), Pair(Symbol("a"), Pair(Pair(Symbol("+"), Pair(1, Pair(2, EmptyList()))), EmptyList())))

In [284]:
read_eval_print("""(hash-set! scheme:expand-table 'set! (lambda args (apply expand-set! args)))""")
read_eval_print("""
(define expand-set!
    (fn (expression environment)
        `(set! ,(first expression) ,(scheme:expand (second expression) environment))))
""")

None
expand-set!


Symbol("expand-set!")

In [285]:
read_eval_print("""(scheme:expand '(set! a (+ 1 2)) scheme:global-environment)""")

(set! a (+ 1 2))


Pair(Symbol("set!"), Pair(Symbol("a"), Pair(Pair(Symbol("+"), Pair(1, Pair(2, EmptyList()))), EmptyList())))

In [286]:
read_eval_print("""(hash-set! scheme:expand-table 'begin (lambda args (apply expand-begin args)))""")
read_eval_print("""
(define expand-begin
    (fn (expression environment)
        `(begin ,@(map (fn (expression) (scheme:expand expression environment)) expression))))
""")

None
expand-begin


Symbol("expand-begin")

In [287]:
read_eval_print("""(scheme:expand '(begin a b) scheme:global-environment)""")

(begin a b)


Pair(Symbol("begin"), Pair(Symbol("a"), Pair(Symbol("b"), EmptyList())))

In [288]:
read_eval_print("""(hash-set! scheme:expand-table 'lambda (lambda args (apply expand-lambda args)))""")
read_eval_print("""
(define expand-lambda
    (fn (expression environment)
        `(lambda ,(first expression) ,(scheme:expand (second expression) environment))))
""")

None
expand-lambda


Symbol("expand-lambda")

In [289]:
read_eval_print("""(scheme:expand '(lambda expr body) scheme:global-environment)""")

(lambda expr body)


Pair(Symbol("lambda"), Pair(Symbol("expr"), Pair(Symbol("body"), EmptyList())))

In [290]:
read_eval_print("""(hash-set! scheme:expand-table 'macro-lambda (lambda args (apply expand-macro-lambda args)))""")
read_eval_print("""
(define expand-macro-lambda
    (fn (expression environment)
        `(macro-lambda ,(first expression) ,(scheme:expand (second expression) environment))))
""")

None
expand-macro-lambda


Symbol("expand-macro-lambda")

In [291]:
read_eval_print("""(scheme:expand '(macro-lambda expr body) scheme:global-environment)""")

(macro-lambda expr body)


Pair(Symbol("macro-lambda"), Pair(Symbol("expr"), Pair(Symbol("body"), EmptyList())))

In [292]:
read_eval_print("""(hash-set! scheme:expand-table 'quote (lambda args (apply expand-quote args)))""")
read_eval_print("""
(define expand-quote
    (fn (expression environment)
        `(quote ,(first expression))))
""")

None
expand-quote


Symbol("expand-quote")

In [293]:
read_eval_print("""(scheme:expand '(quote expr) scheme:global-environment)""")

'expr


Pair(Symbol("quote"), Pair(Symbol("expr"), EmptyList()))

In [294]:
read_eval_print("""
(define scheme:analyze
    (fn (expression)
        (cond ((symbol? expression) (scheme:analyze-symbol expression))
              ((pair? expression)
               (cond ((hash-contains-key? scheme:analyze-table (first expression))
                      ((hash-ref scheme:analyze-table (first expression)) (rest expression)))
                     (#t (scheme:analyze-procedure-application expression))))
              (#t (scheme:analyze-self-evaluating expression)))))
""")

scheme:analyze


Symbol("scheme:analyze")

In [295]:
read_eval_print("""
(define scheme:analyze-self-evaluating
    (fn (expression)
        (fn (k environment)
            (k expression))))
""")

scheme:analyze-self-evaluating


Symbol("scheme:analyze-self-evaluating")

In [296]:
read_eval_print("""(scheme:eval list 1.0)""")

(1.0)


Pair(1.0, EmptyList())

In [297]:
read_eval_print("""
(define scheme:analyze-symbol
    (fn (expression)
        (fn (k environment) (k (environment-symbol-value environment expression)))))
""")

scheme:analyze-symbol


Symbol("scheme:analyze-symbol")

In [298]:
read_eval_print("""(define scheme:analyze-table (make-hash-table))""")

scheme:analyze-table


Symbol("scheme:analyze-table")

In [299]:
read_eval_print("""(hash-set! scheme:analyze-table 'define (fn args (apply scheme:analyze-define args)))""")

None


In [300]:
read_eval_print("""
(define scheme:analyze-define
    (fn (expression)
        (let ((name (first expression))
              (value-expression (scheme:analyze (second expression))))
            (fn (k environment)
                (value-expression (fn (value)
                                      (environment-define-symbol-value environment name value)
                                      (k name))
                                   environment)))))
""")

scheme:analyze-define


Symbol("scheme:analyze-define")

In [301]:
read_eval_print("""(scheme:eval list '(define foo 42+j))""")
read_eval_print("""(scheme:eval list 'foo)""")

(foo)
(42+1j)


Pair((42+1j), EmptyList())

In [302]:
read_eval_print("""(hash-set! scheme:analyze-table 'set! (fn args (apply scheme:analyze-set! args)))""")

None


In [303]:
read_eval_print("""
(define scheme:analyze-set!
    (fn (expression)
        (let ((name (first expression))
              (value-expression (scheme:analyze (second expression))))
            (fn (k environment)
                (value-expression (fn (value)
                                      (environment-set-symbol-value environment name value)
                                      (k value))
                                   environment)))))
""")

scheme:analyze-set!


Symbol("scheme:analyze-set!")

In [304]:
read_eval_print("""(scheme:eval list '(set! foo 42))""")
read_eval_print("""(scheme:eval list 'foo)""")

(42)
(42)


Pair(42, EmptyList())

In [305]:
read_eval_print("""(hash-set! scheme:analyze-table 'begin (fn args (apply scheme:analyze-begin args)))""")

None


In [306]:
read_eval_print("""
(define scheme:analyze-begin
    (fn (expression)
        (cond ((empty? expression) (error "Expected at least one expression in begin"))
              (#t (let ((expressions (map (fn (expression) (scheme:analyze expression)) expression)))
                    (define evaluate-expressions (fn (k environment expressions)
                                                    (cond ((empty? (rest expressions)) ((first expressions) k environment))
                                                          (#t ((first expressions) 
                                                                  (fn _ (evaluate-expressions k environment (rest expressions)))
                                                                  environment)))))
                    (fn (k environment)
                        (evaluate-expressions k environment expressions)))))))
""")

scheme:analyze-begin


Symbol("scheme:analyze-begin")

In [307]:
read_eval_print("""(scheme:eval list '(begin 3))""")
read_eval_print("""(scheme:eval list '(begin (define foo 1) (set! foo 2) foo))""")

(3)
(2)


Pair(2, EmptyList())

In [308]:
read_eval_print("""(hash-set! scheme:analyze-table 'quote (fn args (apply scheme:analyze-quote args)))""")

None


In [309]:
read_eval_print("""
(define scheme:analyze-quote
    (fn (expression)
        (let ((quoted-expression (first expression)))
            (fn (k environment) (k quoted-expression)))))
""")

scheme:analyze-quote


Symbol("scheme:analyze-quote")

In [310]:
read_eval_print("(scheme:eval list '(quote (the quick brown fox)))")

((the quick brown fox))


Pair(Pair(Symbol("the"), Pair(Symbol("quick"), Pair(Symbol("brown"), Pair(Symbol("fox"), EmptyList())))), EmptyList())

In [311]:
read_eval_print("""(hash-set! scheme:analyze-table 'if (fn args (apply scheme:analyze-if args)))""")
read_eval_print("""(hash-set! scheme:expand-table 'if (fn args (apply scheme:expand-if args)))""")

None
None


In [312]:
read_eval_print("""
(define scheme:expand-if
    (fn (expression environment)
        (let ((test (scheme:expand (first expression) environment))
              (consequent (scheme:expand (second expression) environment))
              (alternative (scheme:expand (third expression) environment)))
            `(if ,test ,consequent ,alternative))))
""")

scheme:expand-if


Symbol("scheme:expand-if")

In [313]:
read_eval_print("""
(define scheme:analyze-if
    (fn (expression)
        (let ((test (scheme:analyze (first expression)))
              (consequent (scheme:analyze (second expression)))
              (alternative (scheme:analyze (third expression))))
            (fn (k environment)
                (test
                    (fn (test-value . _)
                        (if test-value (consequent k environment) (alternative k environment)))
                    environment)))))
""")

scheme:analyze-if


Symbol("scheme:analyze-if")

In [314]:
read_eval_print("""(scheme:eval list `(define #t ,#t))""")
read_eval_print("""(scheme:eval list `(define #f ,#f))""")

(\#t)
(\#f)


Pair(Symbol("#f"), EmptyList())

In [315]:
# TODO: Lock symbols

In [316]:
read_eval_print("""(scheme:eval list '(begin (if #t (define foo "true") (define foo "false")) foo))""")
read_eval_print("""(scheme:eval list '(begin (if #f (define foo "true") (define foo "false")) foo))""")

("true")
("false")


Pair('false', EmptyList())

In [317]:
read_eval_print("""(hash-set! scheme:analyze-table 'lambda (fn args (apply scheme:analyze-lambda args)))""")

None


In [318]:
read_eval_print("""
(define scheme:analyze-lambda
    (fn (expression)
        (let ((parameter (first expression))
              (body (scheme:analyze (second expression))))
            (fn (k environment)
                (k (fn (k . args)
                    (let ((environment (environment-extend environment)))
                        (environment-define-symbol-value environment parameter args)
                        (body k environment))))))))
""")

scheme:analyze-lambda


Symbol("scheme:analyze-lambda")

In [319]:
read_eval_print("""
(scheme:eval
    (fn (proc)
        (proc (fn (values) `(values ,@values)) 1 2 3))
    '(lambda args args))
""")

(values 1 2 3)


Pair(Symbol("values"), Pair(1, Pair(2, Pair(3, EmptyList()))))

In [320]:
read_eval_print("""
(define scheme:analyze-procedure-application
    (fn (expression)
        (let ((expressions (map-reverse (fn (expr) (scheme:analyze expr)) expression)))
            (define eval-list
                (fn (k expressions environment values)
                    (cond ((empty? expressions) (k values))
                          (#t ((first expressions)
                                    (fn (result . _)
                                      (eval-list k (rest expressions) environment (cons result values)))
                                    environment)))))
            (fn (k environment)
                (eval-list
                 (fn (values)
                     (scheme:apply k (first values) (rest values)))
                 expressions environment ())))))
""")

scheme:analyze-procedure-application


Symbol("scheme:analyze-procedure-application")

In [321]:
read_eval_print("""
(define scheme:apply
    (fn (k procedure arguments)
        (if (procedure? procedure)
            (apply procedure (cons k arguments))
            (k (apply procedure arguments)))))
""")

scheme:apply


Symbol("scheme:apply")

In [322]:
read_eval_print("""(scheme:eval list '((lambda args args) 1 2 3))""")

((1 2 3))


Pair(Pair(1, Pair(2, Pair(3, EmptyList()))), EmptyList())

In [323]:
read_eval_print("""(hash-set! scheme:analyze-table 'call-with-current-continuation (fn args (apply scheme:analyze-call-with-current-continuation args)))""")

None


In [324]:
read_eval_print("""
(define scheme:analyze-call-with-current-continuation
    (fn (expression)
        (let ((procedure-expression (scheme:analyze (first expression))))
            (fn (k environment)
                (procedure-expression
                    (fn (procedure . _)
                        (procedure k (fn (_ . args) (apply k args))))
                    environment)))))
""")

scheme:analyze-call-with-current-continuation


Symbol("scheme:analyze-call-with-current-continuation")

In [325]:
# TODO: replace with bootstrapped

read_eval_print("""(begin
(environment-define-symbol-value scheme:global-environment 'eval-python eval-python)
(environment-define-symbol-value scheme:global-environment '#f #f)
(environment-define-symbol-value scheme:global-environment '#t #t)

(scheme:eval list '(begin
(define + (eval-python "op.add"))
(define attrgetter (eval-python "op.attrgetter"))
(define cons (eval-python "Pair"))
(define first (attrgetter "a"))
(define rest (attrgetter "b"))
(define list (lambda args args))

(define type? (eval-python "isinstance"))
(define type-predicate 
    (lambda args
        (begin
            (define type (first args)) 
            (lambda args (type? (first args) type)))))

(define make-symbol (eval-python "Symbol"))

(define pair (eval-python "Pair"))
(define pair? (type-predicate pair))
(define eq? (eval-python "op.is_"))
(define empty? (lambda args (eq? (first args) ()))))))""")

(empty?)


Pair(Symbol("empty?"), EmptyList())

In [326]:
read_eval_print("""(scheme:eval list '(begin
(define add2 #f)
(+ 2 (call-with-current-continuation (lambda args (begin (set! add2 (first args)) 3))))))""")
read_eval_print("""(scheme:eval list '(add2 5))""")
read_eval_print("""(scheme:eval list '(add2 32))""")

(5)
(7)
(34)


Pair(34, EmptyList())

In [327]:
read_eval_print("""(hash-set! scheme:analyze-table 'macro-lambda (fn args (apply scheme:analyze-macro-lambda args)))""")

None


In [328]:
read_eval_print("""
(define scheme:analyze-macro-lambda
    (fn (expression)
        (let ((macro-arguments (first expression))
              (body (scheme:analyze (first (rest expression)))))
            (fn (k environment)
                (k (macro-lambda arguments
                    (let ((k (first arguments))
                          (environment (environment-extend environment)))
                        (environment-define-symbol-value environment macro-arguments (rest arguments))
                        (body k environment))))))))
""")

scheme:analyze-macro-lambda


Symbol("scheme:analyze-macro-lambda")

In [329]:
def keval(string): read_eval_print("(scheme:eval (fn (value . _) value) '(begin " + string + "))")

In [330]:
keval("""
(define positional-spec->definition
    (lambda args
        (begin
            (define parameter (first args))
            (define arguments-name (first (rest args)))
            (list 'if (list 'pair? arguments-name) 
                      (list 'define parameter (list 'pop! arguments-name))
                      (list 'missing-positional-argument-error (list 'quote parameter))))))
                      
(positional-spec->definition 'a '<argument>)
""")

(if (pair? <argument>) (define a (pop! <argument>)) (missing-positional-argument-error 'a))


In [331]:
keval("""
(define optional-spec->definition
    (lambda args
        (begin
            (define spec (first args))
            (define spec-name (first spec))
            (define spec-default (first (rest spec)))
            (define arguments-name (first (rest args)))

            (list 'define spec-name 
                          (list 'if (list 'pair? arguments-name)
                                    (list 'pop! arguments-name)
                                    spec-default)))))
                                    
(optional-spec->definition '(a 42) '<arguments>)
""")

(define a (if (pair? <arguments>) (pop! <arguments>) 42))


In [332]:
keval("""
(define rest-spec->definition
    (lambda args
        (begin
            (define spec (first args))
            (define arguments-name (first (rest args)))
            (list 'define spec arguments-name))))
            
(rest-spec->definition 'rest '<arguments>)
""")

(define rest <arguments>)


In [333]:
keval("""
(define parameter-spec->definition
    (lambda args
        (begin
            (define spec (first args))
            (define arguments-name (first (rest args)))
            (if (pair? spec)
                (optional-spec->definition spec arguments-name)
                (positional-spec->definition spec arguments-name)))))
                
(list 
    (parameter-spec->definition 'a '<arguments>)
    (parameter-spec->definition '(a 42) '<arguments>))
""")

((if (pair? <arguments>) (define a (pop! <arguments>)) (missing-positional-argument-error 'a)) (define a (if (pair? <arguments>) (pop! <arguments>) 42)))


In [334]:
keval("""
(define parameter-specs->definitions
    (lambda args
        (begin
            (define parameters (first args))
            (define arguments-name (first (rest args)))
            (if (pair? parameters) 
                (cons (parameter-spec->definition (first parameters) arguments-name)
                      (parameter-specs->definitions (rest parameters) arguments-name))
                (if (empty? parameters)
                    ()
                    (list (rest-spec->definition parameters arguments-name)))))))

(parameter-specs->definitions '(a b (c 42) . rest) '<arguments>)
""")

((if (pair? <arguments>) (define a (pop! <arguments>)) (missing-positional-argument-error 'a)) (if (pair? <arguments>) (define b (pop! <arguments>)) (missing-positional-argument-error 'b)) (define c (if (pair? <arguments>) (pop! <arguments>) 42)) (define rest <arguments>))


In [335]:
"""
(begin 
 (if (pair? <arguments>) 
     (define a (pop! <arguments>)) 
     (missing-positional-argument-error 'a))
 (if (pair? <arguments>) 
     (define b (pop! <arguments>)) 
     (missing-positional-argument-error 'b)) 
 (define c (if (pair? <arguments>) 
               (pop! <arguments>) 
               42))
 (define rest <arguments>))
"""
None

In [336]:
keval("""
(define fn->lambda
    (lambda args
        (begin
            (define fn-spec (first args))
            (define parameter-specs (first fn-spec))
            (define body (rest fn-spec))
            (define <arguments> (make-symbol "<arguments>"))
            (list 'lambda <arguments> (cons 'begin (cons (cons 'begin (parameter-specs->definitions parameter-specs <arguments>)) body))))))

(fn->lambda '((a (b 42) . rest) a b (list a b rest)))
""")

(lambda <arguments> (begin (begin (if (pair? <arguments>) (define a (pop! <arguments>)) (missing-positional-argument-error 'a)) (define b (if (pair? <arguments>) (pop! <arguments>) 42)) (define rest <arguments>)) a b (list a b rest)))


In [337]:
"""
(lambda <arguments> 
    (begin 
        (begin 
            (if (pair? <arguments>) 
                (define a (pop! <arguments>))
                (missing-positional-argument-error 'a)) 
            (define b (if (pair? <arguments>) 
                          (pop! <arguments>) 
                          42))
            (define rest <arguments>))
        a 
        b 
        (list a b rest)))
"""
None

In [338]:
# TODO: bootstrap/ replace with bootstrapped

keval("""
(define print (eval-python "print"))
(define to-string (eval-python "lisp_string"))
(define display (lambda args (begin (print (to-string (first args))) (first args))))
""")
read_eval_print("""(begin
(define print (eval-python "print"))
(define to-string (eval-python "lisp_string"))
(define display (lambda args (begin (print (to-string (first args))) (first args)))))
""")

display
display


Symbol("display")

In [339]:
# TODO: bootstrap replace

keval("""(define fn (macro-lambda args (fn->lambda args)))""")

fn


In [340]:
read_eval_print("""(environment-define-symbol-value scheme:global-environment 'apply scheme:apply)""")
read_eval_print("""(environment-define-symbol-value scheme:global-environment 'eval scheme:eval)""")

None
None


In [341]:
# TODO: bootstrap replace

keval("""
(define pop!
    (macro-lambda args
        (begin
            (define stack (first args))
            (list 'apply 
                (list 'lambda 'value
                    (list 'begin 
                        (list 'set! stack (list 'rest stack))
                        'value))
                (list 'first stack)))))
""")

pop!


In [342]:
read_eval_print("""(scheme:expand '(pop! stack) scheme:global-environment)""")
None

(apply (lambda value (begin (set! stack (rest stack)) value)) (first stack))


In [343]:
keval("""((fn (a (b 42) . rest) (list a b rest)) 1)""")
keval("""((fn (a (b 42) . rest) (list a b rest)) 1 2 3 4 5 6)""")

(1 42 ())
(1 2 (3 4 5 6))
